In [ ]:
import os, glob
import sys
import pydicom
import random
import re
import scipy
import scipy.misc
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from keras.layers import *
import keras.backend as K
from keras.models import Model, Sequential
from keras.optimizers import Adam


plt.set_cmap('gray')
%matplotlib inline

## Seeding 
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed


IMG_DTYPE = np.float32
SEG_DTYPE = np.uint8

In [ ]:
def imshow(*args,**kwargs):
    cmap = kwargs.get('cmap', 'gray')
    title= kwargs.get('title','')
    if len(args)==0:
        raise ValueError("No images given to imshow")
    elif len(args)==1:
        plt.title(title)
        plt.imshow(args[0], interpolation='none')
    else:
        n=len(args)
        if type(cmap)==str:
            cmap = [cmap]*n
        if type(title)==str:
            title= [title]*n
        plt.figure(figsize=(n*5,10))
        for i in range(n):
            plt.subplot(1,n,i+1)
            plt.title(title[i])
            plt.imshow(args[i], cmap[i])
    plt.show()
    
def normalize_image(img):
    """ Normalize image values to [0,1] """
    min_, max_ = float(np.min(img)), float(np.max(img))
    return (img - min_) / (max_ - min_)

In [ ]:
"""
- Author IBBM
- Date 1/3/2019 (DD/MM/YYYY)
- Link https://github.com/IBBM/Cascaded-FCN
"""
def step1_preprocess_img_slice(img_slc):
    """
    Preprocesses the image 3d volumes by performing the following :
    1- Set pixels with hounsfield value great than 1200, to zero.
    2- Clip all hounsfield values to the range [-100, 400]
    3- Apply Histogram Equalization
    """    
    img_slc[img_slc>1200] = 0
    img_slc   = np.clip(img_slc, -100, 400)
    img_slc = normalize_image(img_slc)

    
    img_slc = img_slc * 255
    img_slc = img_slc.astype('uint8')
    img_slc = cv2.equalizeHist(img_slc)
    img_slc = normalize_image(img_slc)
    return img_slc

In [ ]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=128):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        
    def __load__(self, id_name):
        patient_id = id_name.split('_')
        image_path = os.path.join(self.path,"patients", id_name)
        mask_path = os.path.join(self.path,"masks")
        all_masks = os.listdir(mask_path)
        dicom_image = pydicom.dcmread(image_path)
        image = step1_preprocess_img_slice(dicom_image.pixel_array)
        image = normalize_image(image)
        image = np.array(Image.fromarray(image).resize([image_size, image_size])).astype(IMG_DTYPE)
        
        mask = pydicom.dcmread(os.path.join(mask_path,patient_id[0]+'_liver' , id_name)).pixel_array
        mask = mask/255.0
        mask = np.clip(mask, 0, 1)
        mask = np.array(Image.fromarray(mask).resize([image_size, image_size])).astype(IMG_DTYPE)
        mask = mask[:, :, np.newaxis]
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]

        image = []
        mask  = []
    
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            _img = np.stack((_img,)*3, axis=-1)
            image.append(_img)
            mask.append(_mask)
        
        image = np.array(image)
        mask  = np.array(mask)

        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))
    


In [ ]:
image_size = 256
train_path = "train"
batch_size = 8
epochs = 20

## Training Ids
images = []
for file in os.listdir(os.path.join(train_path, "patients")):
        images.append(file)
print(len(images))

val_data_size = len(images)//5

valid_ids = images[:val_data_size]
train_ids = images[val_data_size:]

In [ ]:
gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)

In [ ]:
r = random.randint(0, len(x)-1)
fig = plt.figure(figsize=(20,20))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(x[r])
print(r)
ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(y[r], (image_size, image_size)), cmap="gray")

In [ ]:
# Define the UNet model
def unet_model():
    inputs = Input(x[r].shape)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottle Neck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
    
    # Decoder
    upconv1 = Conv2DTranspose(512, 3, strides=(2, 2), padding='same')(conv5)
    concat1 = Concatenate()([upconv1, conv4])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(concat1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    upconv2 = Conv2DTranspose(256, 3, strides=(2, 2), padding='same')(conv6)
    concat2 = Concatenate()([upconv2, conv3])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(concat2)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    upconv3 = Conv2DTranspose(128, 3, strides=(2, 2), padding='same')(conv7)
    concat3 = Concatenate()([upconv3, conv2])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(concat3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    upconv4 = Conv2DTranspose(64, 3, strides=(2, 2), padding='same')(conv8)
    concat4 = Concatenate()([upconv4, conv1])
    conv9 = Conv2D(64, 3, activation='relu')(concat4)
    conv9 = Conv2D(64, 3, activation='relu')(conv9)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    return Model(inputs=inputs, outputs=outputs)

# Define the BADNet model
def badnet_model():
    inputs = Input(x[r].shape)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottle Neck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    return Model(inputs=inputs, outputs=[conv1, conv2, conv3, conv4, conv5])

# Combine the UNet and BADNet models
def combined_model():
    unet = unet_model()
    badnet = badnet_model()
    
    # Freeze the weights of the BADNet model
    for layer in badnet.layers:
        layer.trainable = False
    
    # Get the outputs of the BADNet model at each encoder block
    badnet_outputs = badnet(unet.inputs)
    
    # Pass the outputs through the skip connections to the decoder blocks of UNet
    upconv1 = Conv2DTranspose(512, 3, strides=(2, 2), padding='same')(badnet_outputs[4])
    concat1 = Concatenate()([upconv1, unet.layers[10].output])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(concat1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
          
    upconv2 = Conv2DTranspose(256, 3, strides=(2, 2), padding='same')(conv6)
    concat2 = Concatenate()([upconv2,unet.layers[8].output])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(concat2)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    upconv3 = Conv2DTranspose(128, 3, strides=(2, 2), padding='same')(conv7)
    concat3 = Concatenate()([upconv3, unet.layers[4].output])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(concat3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    upconv4 = Conv2DTranspose(64, 3, strides=(2, 2), padding='same')(conv8)
    concat4 = Concatenate()([upconv4, unet.layers[2].output])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(concat4)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    return Model(inputs=unet.inputs, outputs=outputs)

In [ ]:
model=combined_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

In [ ]:
gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)

train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, train_path, image_size=image_size, batch_size=batch_size)

train_steps = len(train_ids) // batch_size
valid_steps = len(valid_ids) // batch_size

model.fit(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, epochs=10)

In [ ]:
model.save('models/liver_model4.h5')
model.save_weights("models/liver_weights4.h5")

In [ ]:
model = keras.models.load_model('models/liver_model4.h5', compile= False)
valid_gen = DataGen(valid_ids, train_path, image_size=image_size, batch_size=batch_size)

In [ ]:
x, y = valid_gen.__getitem__(7)
result = model.predict(x)

result = result > 0.5
print(x.shape, result.shape)

imshow(x[1])
imshow(result[1])

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(y[1]*255, (image_size, image_size)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[1]*255, (image_size, image_size)), cmap="gray")

In [ ]:
for i in range(1, 5, 1):
    ## Dataset for prediction
    x, y = valid_gen.__getitem__(i)
    result = model.predict(x)
    result = result > 0.4
    
    for i in range(len(result)):
        fig = plt.figure(figsize=(20,20))
        fig.subplots_adjust(hspace=0.4, wspace=0.4)

        ax = fig.add_subplot(1, 2, 1)
        ax.imshow(np.reshape(y[i]*255, (image_size, image_size)), cmap="gray")
#         ax.imshow(x[i])
        ax = fig.add_subplot(1, 2, 2)
        ax.imshow(np.reshape(result[i]*255, (image_size, image_size)), cmap="gray")

In [ ]:
from sklearn.metrics import confusion_matrix

random_batch = random.randint(0, len(valid_ids)//batch_size - 1)
random_sample = random.randint(0, batch_size-1)
random_batch = 4
random_sample = 2
print(random_batch)
print(random_sample)
x, y = valid_gen.__getitem__(random_batch)
result =  model.predict(x)
result = result > 0

fig2 = plt.figure(figsize=(6,6))
ax2 = fig2.add_subplot(1,1,1)
ax2.imshow(x[random_sample])

fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(y[random_sample]*255, (image_size, image_size)), cmap="gray")
#         ax.imshow(y[i])
ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[random_sample]*255, (image_size, image_size)), cmap="gray")

# imshow(np.reshape(y[random_sample]*255, (image_size, image_size)), cmap="gray")
cm_2d = confusion_matrix(y[random_sample].flatten(), result[random_sample].flatten())
cm = cm_2d.ravel()

# (tn, fp, fn, tp)
print(cm_2d)
print(cm)

In [ ]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = Flatten()
    y_true_flatten = y_true_f(y_true)
    y_pred_f = Flatten()
    y_pred_flatten = y_pred_f(y_pred)
    intersection = tf.reduce_sum(y_true_flatten * y_pred_flatten)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_flatten) + tf.reduce_sum(y_pred_flatten) + smooth)

In [ ]:
print("Pixel Accuracy " + str(((cm[3]+cm[0])/(cm[3]+cm[0]+cm[1]+cm[2])*100))+'%' )
print("True Positive Accuracy " + str(((cm[3])/(cm[3]+cm[2])*100))+'%' )
# print("Dice Coefficient " + str(dice_coef(y[random_sample], result[random_sample])))


In [ ]:
import seaborn as sns
ax= plt.subplot()
cm_2d = cm_2d.astype('float') / cm_2d.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_2d, annot=True, ax = ax); #annot=True to annotate cells

# print(count_cms)
# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['No Tumor', 'Tumor']); ax.yaxis.set_ticklabels(['No Tumor', 'Tumor']);